In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Semisupervised GNN to classify nodes where:
## paper == node
## citations == edges

# Each paper node contains a word count vector as features, normalized so that they sum up to one

In [3]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories: ', dataset.num_classes)

Extracting file to /home/sodiq/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories:  7


In [5]:
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})

In [6]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [27]:
# Building a two-layer GCN. Each layer computes new node representations by aggregating neighbor information

from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feats):
        h = F.relu(self.conv1(g, in_feats))
        h = self.conv2(g, h)
        return h

# create the model with given dimension
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes) # [2708, 16, 7]

In [28]:
epochs = 500

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(epochs):
        # forward
        logits = model(g, features)

        # compute prediction
        pred = logits.argmax(1)

        # compute loss (only training set)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 50 == 0:
            print(f'In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})')

train(g, model)



In epoch 0, loss: 1.946, val acc: 0.186 (best 0.186), test acc: 0.204 (best 0.204)
In epoch 50, loss: 0.577, val acc: 0.752 (best 0.752), test acc: 0.770 (best 0.770)
In epoch 100, loss: 0.085, val acc: 0.772 (best 0.776), test acc: 0.778 (best 0.779)
In epoch 150, loss: 0.030, val acc: 0.774 (best 0.776), test acc: 0.770 (best 0.779)
In epoch 200, loss: 0.016, val acc: 0.776 (best 0.780), test acc: 0.760 (best 0.764)
In epoch 250, loss: 0.010, val acc: 0.776 (best 0.780), test acc: 0.762 (best 0.764)
In epoch 300, loss: 0.007, val acc: 0.770 (best 0.780), test acc: 0.763 (best 0.764)
In epoch 350, loss: 0.005, val acc: 0.772 (best 0.780), test acc: 0.763 (best 0.764)
In epoch 400, loss: 0.004, val acc: 0.772 (best 0.780), test acc: 0.764 (best 0.764)
In epoch 450, loss: 0.003, val acc: 0.770 (best 0.780), test acc: 0.763 (best 0.764)


In [30]:
# Training on GPU
# g = g.to('cuda')
# model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
# train(g, model)